In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numba

In [2]:
def b_rules(lasR, LasR, lasI, LasI, AI1, LasRAI1):
    
    k_lasR = 0.004
    g_lasR = 0.002
    k_LasR = 0.4
    g_LasR = 0.35
    a_lasI = 0.00036
    b_lasI = 0.0058
    K1 = 1.2
    h1 = -1.4
    g_lasI = 0.001
    k_LasI = 0.7
    g_LasI = 0.12
    k_AI1 = 1
    g_AI1 = 0.3
    u_LasRAI1 = 0.05
    s_LasRAI1 = 0.25
    g_LasRAI1 = 0.14
    # Cell parameters
    d = 0.8
    
    S0 = k_lasR                                       # + lasR
    S1 = lasR*g_lasR                                  # - lasR
    S2 = lasR*k_LasR                                  # + LasR
    S3 = LasRAI1*s_LasRAI1                            # + LasR ; + AI1 ; - LasRAI1
    S4 = AI1*LasR*u_LasRAI1                           # - LasR ; - AI1 ; + LasRAI1
    S5 = LasR*g_LasR                                  # - LasR
    S6 = a_lasI + (b_lasI/(1+(LasRAI1/K1)**h1))       # + lasI
    S7 = lasI*g_lasI                                  # - lasI
    S8 = lasI*k_LasI                                  # + LasI
    S9 = LasI*g_LasI                                  # - LasI
    S10 = LasI*k_AI1                                  # + AI1
    S11 = AI1*g_AI1                                   # - AI1
    S12 = d*AI1                                       # - AI1 ; + AI1_ext
    S13 = LasRAI1*g_LasRAI1                           # - LasRAI1
    
    Stot = S0 + S1 + S2 + S3 + S4 + S5 + S6 + S7 + S8 + S9 + S10 + S11 + S12 + S13
    
    U = np.random.rand()
    Tau = np.random.exponential(scale=1/Stot)
    
    if (U < S0/Stot):
        return np.array((U, Tau, 0), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U < (S0+S1)/Stot):
        return np.array((U, Tau, 1), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U < (S1+S2+S3)/Stot):
        return np.array((U, Tau, 2), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3)/Stot):
        return np.array((U, Tau, 3), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4)/Stot):
        return np.array((U, Tau, 4), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5)/Stot):
        return np.array((U, Tau, 5), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6)/Stot):
        return np.array((U, Tau, 6), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7)/Stot):
        return np.array((U, Tau, 7), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8)/Stot):
        return np.array((U, Tau, 8), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8+S9)/Stot):
        return np.array((U, Tau, 9), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8+S9+S10)/Stot):
        return np.array((U, Tau, 10), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8+S9+S10+S11)/Stot):
        return np.array((U, Tau, 11), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    elif (U <(S0+S1+S2+S3+S4+S5+S6+S7+S8+S9+S10+S11+S12)/Stot):
        return np.array((U, Tau, 12), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    else:
        return np.array((U, Tau, 13), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])

In [3]:
def e_rules(AI1_ext):
    
    g_AI1_ext = 0.8
    d = 0.8
    d_away = 1.2
    
    S14 = AI1_ext*d                       # + AI1 ; - AI1_ext
    S15 = AI1_ext*(d_away + g_AI1_ext)    # - AI1_ext
    
    Stot = S14 + S15
    
    U = np.random.rand()
    Tau = np.random.exponential(scale=1/Stot)
    
    if (U < S14/Stot):
        return np.array((U, Tau, 14), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])
    else:
        return np.array((U, Tau, 15), dtype=[('u','f4'), ('tau','f4'), ('j','i4')])

In [4]:
def rule_application(N, bacteria, env, reactions_sorted):
    
    if reactions_sorted['i'] > N:
        if reactions_sorted['j'] == 14:
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = np.random.randint(0,N+1)
            bacteria[i,4,t+1] = bacteria[i,4,t]+1
            env[t+1] = env[t]-1
        else:
            bacteria[:,:,t+1] = bacteria[:,:,t]
            env[t+1] = env[t]-1

    else:
        if reactions_sorted['j'] == 0:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,0,t+1] = bacteria[i,0,t]+1
        elif reactions_sorted['j'] == 1:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,0,t+1] = bacteria[i,0,t]-1
        elif reactions_sorted['j'] == 2:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,1,t+1] = bacteria[i,1,t]+1
        elif reactions_sorted['j'] == 3:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,1,t+1] = bacteria[i,1,t]+1
            bacteria[i,4,t+1] = bacteria[i,4,t]+1
            bacteria[i,5,t+1] = bacteria[i,5,t]-1
        elif reactions_sorted['j'] == 4:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,1,t+1] = bacteria[i,1,t]-1
            bacteria[i,4,t+1] = bacteria[i,4,t]-1
            bacteria[i,5,t+1] = bacteria[i,5,t]+1
        elif reactions_sorted['j'] == 5:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,1,t+1] = bacteria[i,1,t]-1
        elif reactions_sorted['j'] == 6:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,2,t+1] = bacteria[i,2,t]+1
        elif reactions_sorted['j'] == 7:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,2,t+1] = bacteria[i,2,t]-1
        elif reactions_sorted['j'] == 8:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,3,t+1] = bacteria[i,3,t]-1
        elif reactions_sorted['j'] == 9:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,3,t+1] = bacteria[i,3,t]-1
        elif reactions_sorted['j'] == 10:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,4,t+1] = bacteria[i,4,t]+1
        elif reactions_sorted['j'] == 11:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,4,t+1] = bacteria[i,4,t]-1
        elif reactions_sorted['j'] == 12:
            env[t+1] = env[t]+1
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,4,t+1] = bacteria[i,4,t]+1
        else:
            env[t+1] = env[t]
            bacteria[:,:,t+1] = bacteria[:,:,t]
            i = reactions_sorted['i']
            bacteria[i,5,t+1] = bacteria[i,5,t]-1

In [6]:
def multicompartmental_gillespie(N, t_max, b_0, e_0):
    
    steps = t_max/100
    
    # Initial conditions
    t = np.zeros(steps)
    b_0 = np.array((0, 0, 0, 0, 0, 0))   # (lasR_0, LasR_0, lasI_0, LasI_0, AI1_0, LasRAI1_0)
    e_0 = np.array((0))                  # (AI1_ext_0)
    
    # Simulation initialization
    bacteria = np.zeros(N, len(b_0), steps)
    i = 0
    x = 0
    while i < N:
        while x < len(b_0):
            bacteria[i,x,0] = b_0[x]
            x += 1
        i += 1

    env = np.zeros(len(e_0), steps)
    x = 0
    while x < len(e_0):
        env[x,0] = e_0[x]
    
    # Simulation
    step = 0
    i = 0
    while step < steps-1:
        reactions = np.zeros(N+1, dtype=[('tau','f4'), ('j','i4'), ('i','i4')])
        while i < N:
            reactions[i]['tau'] = b_rules(bacteria[i,:,step])['tau']
            reactions[i]['j'] = b_rules(bacteria[i,:,step])['j']
            reactions[i]['i'] = i
            i += 1
        reactions[N+1]['tau'] =  e_rules(env[step])['tau']
        reactions[N+1]['j'] =  e_rules(env[step])['j']
        reactions[N+1]['i'] =  N+1
        
        reactions_sorted = np.sort(reactions, order='tau')
    
    t[step+1] = reactions_sorted[0]['tau']
    rule_application(N, bacteria, env, reactions_sorted)
        
    step += 1